In [1]:
import torch as t
import nnsight
from nnsight import LanguageModel, LanguageModel
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/aansh/nnsightusage/nns2env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = LanguageModel("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", device_map="auto")
print(llm)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [3]:
def rearrange_activations(file_path: str):
    out_dict = t.load(file_path)
    final_array = t.empty(1)
    for i in range(len(llm.model.layers)):
        if i == 0:
            final_array = t.cat(out_dict[f"layer_{i}"], dim=1)
        else:
            prompt_array = t.cat(out_dict[f"layer_{i}"], dim=1)
            final_array = t.cat([final_array, prompt_array], dim=0)

    final_array = t.cat([final_array[:, 0:1, :], final_array[:, 20:, :]], dim=1)
    return final_array

In [4]:
def find_first_chinese_token(file_path: str):
    # Define a regex pattern for Chinese characters (Unicode range for CJK Unified Ideographs)
    chinese_pattern = re.compile(r'[\u4e00-\u9fff]+')

    # Read the text from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Tokenize the text
    tokens = llm.tokenizer(text)
    # Find the first token containing Chinese characters
    for idx, token_id in enumerate(tokens["input_ids"]):
        if chinese_pattern.search(llm.tokenizer.decode(token_id)):
            return llm.tokenizer.decode(token_id), token_id, idx  # Return the first Chinese token found

    return None  # Return None if no Chinese token is found

In [5]:
# # Assume `activation_probability` is the computed (28, 1536) matrix
# plt.figure(figsize=(15, 6))  # Adjust figure size for better visibility

# # Plot the heatmap
# sns.heatmap(thresholded_tensor.numpy(), cmap="jet", xticklabels=False, yticklabels=True)

# # Labels and title
# plt.xlabel("Neuron Index")
# plt.ylabel("Layer Index")
# # plt.title("Activation Probability Heatmap (Above 95th Percentile)")

# # Show the plot
# plt.show()


In [6]:
# colorscale = [
#     [0.0, "black"],  # 0 maps to white
#     [1e-10, "rgb(68, 1, 84)"],  # Start of Viridis colormap
#     [1.0, "rgb(253, 231, 37)"]  # End of Viridis colormap
# ]

# # Step 3: Plot the heatmap
# fig = go.Figure(data=go.Heatmap(
#     z=(thresholded_tensor.T/thresholded_tensor.T.flatten().max()),
#     colorscale=colorscale,
#     colorbar=dict(title='Normalized Value'),
#     zmin=0,  # Ensure 0 is mapped to white
#     zmax=1   # Ensure normalization is respected
# ))

# # Update layout for better visualization
# fig.update_layout(
#     title='>95 Percentile Neuron Activations',
#     xaxis=dict(title='Layer Number'),
#     yaxis=dict(title='Neuron'),
#     width=800,
#     height=600,
#     template="plotly_dark"
# )

# # Show the plot
# fig.show()

In [7]:
# num_non_zero = t.count_nonzero(thresholded_tensor)
# num_non_zero/(28*1536)

In [8]:
def get_activated_neuron_indices(llm: LanguageModel,
                                 ans_dir_path: str, 
                                 act_dir_path: str,
                                 k_percentile: float = 0.95):
    
    ans_dir = os.listdir(ans_dir_path)
    act_dir = os.listdir(act_dir_path)
    act_dir.sort()
    ans_dir.sort()
    act_ls = []

    for i, act in enumerate(act_dir):
        print(f"File: {i}")
        act_path = os.path.join(act_dir_path, act)
        ans_path = os.path.join(ans_dir_path, ans_dir[i])
        print(f"act_path: {act_path}")
        print(f"ans_path: {ans_path}")
        fa = rearrange_activations(act_path)
        ch_token, token_id, token_idx = find_first_chinese_token(ans_path)
        print(f"ch_token: {ch_token}")
        act_ls.append(fa[:, token_idx, :])
        
    activations = t.stack(act_ls, dim=0)
    expected_activations = activations.mean(dim=0)
    threshold = t.quantile(expected_activations, k_percentile)
    thresholded_tensor = t.where(expected_activations >= threshold, expected_activations, t.tensor(0.0))
    return thresholded_tensor

In [9]:
ans_dir_path = "/home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Answers/Chinese"
act_dir_path = "/home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese"
activationss = get_activated_neuron_indices(llm, ans_dir_path, act_dir_path, k_percentile=0.99)

File: 0
act_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese/act_run_1.pth
ans_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Answers/Chinese/ans_run_1.txt
ch_token: 要
File: 1
act_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese/act_run_16.pth
ans_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Answers/Chinese/ans_run_16.txt
ch_token: 要
File: 2
act_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese/act_run_21.pth
ans_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Answers/Chinese/ans_run_21.txt
ch_token: 要
File: 3
act_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese/act_run_23.pth
ans_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Answers/Chinese/ans_run_23.txt
ch_token: 要
File: 4
act_path: /home/aansh/nnsightusage/Logs/Partitioned_Cached_Runs_1/Activations/Chinese/act_run_24.pth
ans_path: /home/aansh

In [10]:
# import pandas as pd

# import plotly.express as px

# # Convert the activations tensor to a DataFrame for easier plotting
# activations_np = activationss.numpy()
# num_layers, num_neurons = activations_np.shape

# # Create a DataFrame with layer, neuron index, and activation value
# data = {
#     'Layer': [],
#     'Neuron': [],
#     'Activation': []
# }

# for layer in range(num_layers):
#     for neuron in range(num_neurons):
#         data['Layer'].append(layer)
#         data['Neuron'].append(neuron)
#         data['Activation'].append(activations_np[layer, neuron])

# df = pd.DataFrame(data)

# # Filter out zero activations for better visualization
# df = df[df['Activation'] > 0]

# # # Create the scatter plot
# # fig = px.scatter(df, x='Neuron', y='Layer', size='Activation', color='Activation',
# #                  color_continuous_scale='Viridis', title='Active Neurons in LLM',
# #                  labels={'Neuron': 'Neuron Index', 'Layer': 'Layer Index', 'Activation': 'Activation Value'})

# # # Update layout for better visualization
# # fig.update_layout(height=800, width=1200)
# # Create the scatter plot with Layer on the x-axis

# fig = px.scatter(df, x='Layer', y='Neuron', size='Activation', color='Activation',
#                  color_continuous_scale='Viridis', title='Active Neurons in LLM',
#                  labels={'Neuron': 'Neuron Index', 'Layer': 'Layer Index', 'Activation': 'Activation Value'})

# # Update layout for better visualization
# fig.update_layout(height=500, width=1000)
# fig.show()

# # Customizing the scatter plot with additional features



In [11]:
# import plotly.graph_objects as go

# # Create a list of dropdown options for each layer
# dropdown_buttons = [
#     {
#         'label': f'Layer {i}',
#         'method': 'update',
#         'args': [{'visible': [j == i for j in range(num_layers)]},
#                  {'title': f'Active Neurons in Layer {i}'}]
#     }
#     for i in range(num_layers)
# ]

# # Create a list of traces, one for each layer
# traces = []
# for i in range(num_layers):
#     layer_df = df[df['Layer'] == i]
#     trace = go.Scatter(
#         x=layer_df['Neuron'],
#         y=layer_df['Activation'],
#         mode='markers',
#         marker=dict(size=layer_df['Activation'], color=layer_df['Activation'], colorscale='Viridis'),
#         name=f'Layer {i}',
#         visible=(i == 0)  # Only the first layer is visible initially
#     )
#     traces.append(trace)

# # Create the figure with the traces and add the dropdown menu
# fig = go.Figure(data=traces)
# fig.update_layout(
#     updatemenus=[{
#         'buttons': dropdown_buttons,
#         'direction': 'down',
#         'showactive': True,
#     }],
#     title='Active Neurons in Layer 0',
#     xaxis_title='Neuron Index',
#     yaxis_title='Activation Value',
#     height=500,
#     width=1000
# )

# fig.show()

In [12]:
def save_text_to_file(text: dict,
                      filename:str,
                      dir_path: str):
    
    os.makedirs(dir_path, exist_ok=True)  
    file_path = os.path.join(dir_path, filename)  
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text)
    print(f"File saved at: {file_path}")
    
def save_acts_to_file(hidden_states: dict,
                      filename: str,
                      dir_path: str):

    os.makedirs(dir_path, exist_ok=True)
    file_path = os.path.join(dir_path, filename)
    t.save(hidden_states, file_path)
    print(f"File saved at: {file_path}")
    

In [13]:
prompt = """
Prove that the sum of first n odd natural numbers is equal to n^2.
"""

In [14]:
def generate_output(llm: LanguageModel,
                    prompt:str,
                    append_eos_token: bool = True,
                    n_new_tokens:int = 200,
                    temperature: float = 1,
                    print_ans: bool = True,
                    save_ans: bool = False, 
                    ans_filename: str = "ans.txt",
                    ans_dir_path: str = "/home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers",
                    save_activations: bool = False,
                    act_filename: str = "acts.pth",
                    act_dir_path: str = "/home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations", 
                    thresholded_tensor: t.Tensor = None):
    
    if append_eos_token:
        prompt += llm.tokenizer.eos_token
    
    with llm.generate(prompt, max_new_tokens=n_new_tokens, temperature=temperature) as tracer:
        hidden_states = nnsight.dict().save()
        for i in range(len(llm.model.layers)):
            hidden_states[f"layer_{i}"] = []
        llm.model.layers.all()
        nz_x = t.nonzero(thresholded_tensor)
        nz_x = [(nz[0].item(), nz[1].item()) for nz in nz_x]
        for i in range(len(llm.model.layers)):
            for j in range(1536):
                if (i, j) in nz_x:
                    llm.model.layers[i].mlp.output[:, -1, j] = thresholded_tensor[i, j]
                
        for i in range(len(llm.model.layers)):
            hidden_states[f"layer_{i}"].append(llm.model.layers[i].mlp.output.cpu())
        out = llm.generator.output.cpu().save()
        
    decoded_prompt = llm.tokenizer.decode(out[0][0:50])
    decoded_answer = llm.tokenizer.decode(out[0][-n_new_tokens:])
    
    if print_ans:
        print(f"Prompt: {decoded_prompt}")
        print(f"Generated Answer: {decoded_answer}")
        
    if save_ans:
        save_text_to_file(decoded_answer, ans_filename, ans_dir_path)
        
    if save_activations:
        save_acts_to_file(hidden_states, act_filename, act_dir_path)

In [15]:
for i in range(3, 100):
    ans_filename = f"ans_run_{i+1}.txt"
    act_filename = f"act_run_{i+1}.pth"
    generate_output(llm,
                    prompt,
                    print_ans=False,
                    save_ans=True,
                    ans_filename=ans_filename,
                    save_activations=True,
                    act_filename=act_filename,
                    thresholded_tensor=activationss)
    

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers/ans_run_4.txt
File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations/act_run_4.pth


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers/ans_run_5.txt
File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations/act_run_5.pth


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers/ans_run_98.txt
File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations/act_run_98.pth


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers/ans_run_99.txt
File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations/act_run_99.pth


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Answers/ans_run_100.txt
File saved at: /home/aansh/nnsightusage/Logs/Patched_Runs_1/Activations/act_run_100.pth
